In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from scipy import stats as st
import math as mth
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set(rc={'figure.figsize':(16, 9)})
from plotly import graph_objects as go
import plotly.express as px

In [3]:
df = pd.read_csv('logs_exp.csv', sep='\t')

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
df.head()

,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       244126 non-null  object
 1   DeviceIDHash    244126 non-null  int64 
 2   EventTimestamp  244126 non-null  int64 
 3   ExpId           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


In [6]:
#переименовка колонок
df.rename(columns = {'EventName':'event_name', 'DeviceIDHash':'user_id', 'EventTimestamp':'event_timestamp', 'ExpId':'exp_id'}, inplace = True)
df.sample()

,event_name,user_id,event_timestamp,exp_id
86167,MainScreenAppear,6838581104592469621,1564828596,247


In [7]:
#приведение к верному типу данных
df['event_timestamp'] = df['event_timestamp'].astype('datetime64[s]')
df.sample()

,event_name,user_id,event_timestamp,exp_id
117445,MainScreenAppear,8459733975219661242,2019-08-04 09:56:49,248


In [8]:
#проверка на дубликаты
df.duplicated().sum()

413

In [9]:
#удаление дубликатов
df = df.drop_duplicates()

In [10]:
#добавление столбца с обозначением контрольных групп и эксперементальной А1, А2 и В
df['test_group'] = df['exp_id'].replace(246, 'A1')
df['test_group'] = df['test_group'].replace(247, 'A2')
df['test_group'] = df['test_group'].replace(248, 'B')

In [11]:
df.head(5)

,event_name,user_id,event_timestamp,exp_id,test_group
0,MainScreenAppear,4575588528974610257,2019-07-25 04:43:36,246,A1
1,MainScreenAppear,7416695313311560658,2019-07-25 11:11:42,246,A1
2,PaymentScreenSuccessful,3518123091307005509,2019-07-25 11:28:47,248,B
3,CartScreenAppear,3518123091307005509,2019-07-25 11:28:47,248,B
4,PaymentScreenSuccessful,6217807653094995999,2019-07-25 11:48:42,248,B


In [12]:
users = (df.groupby('test_group').agg({'user_id':'nunique'})
      .reset_index()
      .sort_values(by='user_id', ascending=False)
     )
users

,test_group,user_id
2,B,2542
1,A2,2520
0,A1,2489


> Есть пользователи каждой из трех групп.
>
> Необходимо проверить, нет ли пользователей, находящихся более чем в 1 группе.


In [13]:

#проверим группы A1 и A2
len(np.intersect1d(df.query('test_group == "A1"')['user_id'].unique(), df.query('test_group == "A2"')['user_id'].unique()))

0

In [14]:
#проверим группы A1 и B
len(np.intersect1d(df.query('test_group == "A1"')['user_id'].unique(), df.query('test_group == "B"')['user_id'].unique()))

0

In [15]:
#проверим группы A2 и B
len(np.intersect1d(df.query('test_group == "А2"')['user_id'].unique(), df.query('test_group == "B"')['user_id'].unique()))

0

> **Вывод**
>
> Во всех группах присутствуют уникальные пользователи (один пользователь не состоит в нескольких группах).

In [16]:
#посмотрим, сколько раз встречается каждое событие
event_count = df['event_name'].value_counts().reset_index()
event_count

,index,event_name
0,MainScreenAppear,119101
1,OffersScreenAppear,46808
2,CartScreenAppear,42668
3,PaymentScreenSuccessful,34118
4,Tutorial,1018


In [17]:
#посмотрим, сколько пользователей совершали каждое событие
event_counts_users = df.groupby('event_name')['user_id'].nunique().sort_values(ascending=False).to_frame().reset_index()\
        .rename(columns={'user_id': 'users_count'}) 
event_counts_users

,event_name,users_count
0,MainScreenAppear,7439
1,OffersScreenAppear,4613
2,CartScreenAppear,3749
3,PaymentScreenSuccessful,3547
4,Tutorial,847


In [18]:
all_events_count = df.pivot_table(index='event_name', columns='test_group',values='user_id',aggfunc='nunique')\
                       .sort_values('A1',ascending=False)
all_events_count['A1+A2'] = all_events_count['A1'] + all_events_count['A2']
all_events_count.drop(['Tutorial'], inplace=True)
all_events_count

test_group,A1,A2,B,A1+A2
event_name,,,,
MainScreenAppear,2456,2482,2501,4938
OffersScreenAppear,1545,1530,1538,3075
CartScreenAppear,1270,1240,1239,2510
PaymentScreenSuccessful,1202,1160,1185,2362


Добавим в таблицу с числом пользователей по группам значение общей грппы объединяющей контрольные группы А1 и А2

In [19]:
users.loc[3] = ['A1+A2', 4997]
users = users.set_index(users.columns[0])
users

,user_id
test_group,
B,2542
A2,2520
A1,2489
A1+A2,4997


Для проверки гипотез воспользуемся **Z-критерием** и применим **метод Бонферрони (поправка Бонферрони)**, так как проводим множественное сравнение. В нашем случае мы будем проводить 16 сравнений, следовательно, bonferroni_alpha будет равно alpha / 16.

Сформулируем гипотезы:

**H0**: Конверсия пользователей в обеих группах равна

**H1**: Конверсия в обеих группах не равна

In [20]:
#функцию для Z-критерия
def print_z_test_with_bonferroni_validation_result(group1, group2, event, alpha): 
    p1_event = all_events_count.loc[event, group1]
    p2_event = all_events_count.loc[event, group2] 
    p1_users = users.loc[group1, 'user_id'] 
    p2_users = users.loc[group2, 'user_id'] 
    p1 = p1_event / p1_users 
    p2 = p2_event / p2_users 
    difference = p1 - p2
    p_combined = (p1_event + p2_event) / (p1_users + p2_users) 
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / p1_users + 1 / p2_users))
    distr = st.norm(0, 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    bonferroni_alpha = alpha / 16
    
    print('Проверка для групп {} и {}, событие: {}, p-значение: {p_value:.2f}'.format(group1, group2, event, p_value=p_value))
    if (p_value < bonferroni_alpha):
        print("Отвергаем нулевую гипотезу о равенстве конверсии в группах")
    else:
        print("Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах")

**А/А Тест**

In [21]:
for event in all_events_count.index:
    print_z_test_with_bonferroni_validation_result('A1', 'A2', event, 0.05)
    print()

Проверка для групп A1 и A2, событие: MainScreenAppear, p-значение: 0.59
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A1 и A2, событие: OffersScreenAppear, p-значение: 0.32
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A1 и A2, событие: CartScreenAppear, p-значение: 0.20
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A1 и A2, событие: PaymentScreenSuccessful, p-значение: 0.11
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах



**А/B Тест**

In [22]:
#группы А1 и В

for event in all_events_count.index:
    print_z_test_with_bonferroni_validation_result('A1', 'B', event, 0.05)
    print()

Проверка для групп A1 и B, событие: MainScreenAppear, p-значение: 0.40
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A1 и B, событие: OffersScreenAppear, p-значение: 0.25
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A1 и B, событие: CartScreenAppear, p-значение: 0.11
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A1 и B, событие: PaymentScreenSuccessful, p-значение: 0.23
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах



In [23]:
#группы А2 и В

for event in all_events_count.index:
    print_z_test_with_bonferroni_validation_result('A2', 'B', event, 0.05)
    print()

Проверка для групп A2 и B, событие: MainScreenAppear, p-значение: 0.76
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A2 и B, событие: OffersScreenAppear, p-значение: 0.88
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A2 и B, событие: CartScreenAppear, p-значение: 0.74
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A2 и B, событие: PaymentScreenSuccessful, p-значение: 0.68
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах



In [24]:
#группы А1+А2 и В

for event in all_events_count.index:
    print_z_test_with_bonferroni_validation_result('A1+A2', 'B', event, 0.05)
    print()

Проверка для групп A1+A2 и B, событие: MainScreenAppear, p-значение: 0.12
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A1+A2 и B, событие: OffersScreenAppear, p-значение: 0.38
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A1+A2 и B, событие: CartScreenAppear, p-значение: 0.22
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

Проверка для групп A1+A2 и B, событие: PaymentScreenSuccessful, p-значение: 0.59
Не получилось отвергнуть нулевую гипотезу о равенстве конверсии в группах

